## Functionality: visualization of functional connectivity

<ul>
    <li>142 channels = 1-96 channels (M1) + 97-100(not used) + 101-132(ThaSMA) + 133-146 (GP/STN)</li>
    <li>load animetafolder/pinky_locThaSMA.csv for the details of ThaSMA </li>
</ul>

## Import Packages 

In [1]:
import os, sys
import numpy as np
import pandas as pd

import pickle

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)

# import animal_folder function in util/folder_extract.pynb 
from util.folder_extract import exp_subfolders


# import graphVisual in connAnalyTool/graphVisual.pynb 
from connAnalyTool import graphVisual


from igraph.drawing.text import TextDrawer
import cairo

importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb
importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/connAnalyTool/graphVisual.ipynb


## exp subfolders

In [2]:
datafolder, _, pipelinefolder, _ = exp_subfolders()

## load the ciCOH matrix

### Load Beta band ciCOH
1. dic_ciCOH.keys() = ['ciCOH_beta1_normal', 'ciCOH_beta1_mild', 'ciCOH_beta2_normal', 'ciCOH_beta2_mild']

In [3]:
def ciCOHBetabandLoad(betaciCOHfilefolder):
    """
        load beta band ciCOH
        
        
        @return ciCOH_beta1_normal, ciCOH_beta1_mild: bata1 band ciCOH values in normal and mild conditions
                                                      (ndarray: nchns * nchns * ntrials)
        
        @return ciCOH_beta2_normal, ciCOH_beta2_mild: bata2 band ciCOH values in normal and mild conditions 
                                                      (ndarray: nchns * nchns * ntrials)
        
    """

    # file storing the ciCOH results
    ciCOHfile = os.path.join(betaciCOHfilefolder, 'res_ciCOH_beta.pickle')

    # load res_ciCOH
    res_ciCOH_beta = pickle.load(open(ciCOHfile, "rb" ))

    # ciCOH_beta1_normal, ciCOH_beta1_mild
    ciCOH_beta1_normal, ciCOH_beta1_mild = res_ciCOH_beta['ciCOH_beta1_normal'], res_ciCOH_beta['ciCOH_beta1_mild']

    # ciCOH_beta2_normal, ciCOH_beta2_mild
    ciCOH_beta2_normal, ciCOH_beta2_mild = res_ciCOH_beta['ciCOH_beta2_normal'], res_ciCOH_beta['ciCOH_beta2_mild']
    
    # return
    return ciCOH_beta1_normal, ciCOH_beta1_mild, ciCOH_beta2_normal, ciCOH_beta2_mild

In [4]:
betaciCOHfilefolder = os.path.join(pipelinefolder, '1_dataAnaly', 'FCAnaly', '0_calcciCOH_beta', 'out')

# load beta1/beta2 ciCOH in normal and mild conditions
ciCOH_beta1_normal, ciCOH_beta1_mild, ciCOH_beta2_normal, ciCOH_beta2_mild = ciCOHBetabandLoad(betaciCOHfilefolder)

### Load alpha band ciCOH
1. dic_ciCOH.keys() = ['ciCOH_alpha_normal', 'ciCOH_alpha_mild']

In [5]:
def ciCOHAlphabandLoad(alphaciCOHfilefolder):
    """
        load alpha band ciCOH
        
        @return ciCOH_alpha_normal,  ciCOH_alpha_mild: alpha band ciCOH values in normal and mild 
                                                       (ndarray: nchns * nchns * ntrials) 
    """

    # file storing the ciCOH results
    ciCOHfile = os.path.join(alphaciCOHfilefolder, 'res_ciCOH_alpha.pickle')

    # load res_ciCOH_alpha
    res_ciCOH_alpha = pickle.load(open(ciCOHfile, "rb" ))

    # ciCOH_alpha_normal, ciCOH_alpha_mild
    ciCOH_alpha_normal, ciCOH_alpha_mild = res_ciCOH_alpha['ciCOH_alpha_normal'], res_ciCOH_alpha['ciCOH_alpha_mild']
    
    # return
    return ciCOH_alpha_normal,  ciCOH_alpha_mild

In [6]:
alphaciCOHfilefolder = os.path.join(pipelinefolder, '1_dataAnaly', 'FCAnaly', '0_calcciCOH_alpha', 'out')

# load alpha ciCOH in normal and mild conditions
ciCOH_alpha_normal,  ciCOH_alpha_mild = ciCOHAlphabandLoad(alphaciCOHfilefolder)

## Load Channel Information

load channel information from datafolder/pinky_chnInf.csv

In [7]:
df = pd.read_csv(os.path.join(datafolder, 'pinky_chnInf.csv'))

idx_notused = [100, 104, 109, 120, 124]

# brain area
elec_area = list(df['brainarea'])


# delete the unused channels in elec_area
for idx in reversed(idx_notused):
    
    del elec_area[idx]

In [8]:
x, y = np.asarray(df['x']), np.asarray(df['y'])

# coordinates of all channel
elec_coord = np.concatenate((np.expand_dims(x, axis=1), np.expand_dims(y, axis=1)), axis = 1)
    
# delete the unused channels in elec_coord
elec_coord = np.delete(elec_coord, idx_notused, axis=0)

## Alpha Visual

In [65]:
texts = dict()

texts['right Caudate'] = [250, 30, 20]
texts['VPLo'] = [0, 100, 20]

texts['VLo'] = [0, 155, 20]

texts['VA'] = [0, 225, 20]

texts['left MC'] = [-0, 275, 20]

texts['SMA'] = [0, 345, 20]

texts['M1'] = [-10, 510, 20]

In [9]:
lowweight = 0.1

savepath = os.path.join(pipelinefolder, '1_dataAnaly', 'FCAnalys', '1_visual_FC', 'out')

### normal

In [94]:
del bandciCOH

bandciCOH = ciCOH_alpha_normal[0:128, 0:128, :]

filename = 'ciCOH_alpha_normal' + '.png'

savefile = os.path.join(savepath, filename)

In [95]:
# delete the unused channels in weight
bandciCOH = np.delete(bandciCOH, idx_notused, axis=0)
bandciCOH = np.delete(bandciCOH, idx_notused, axis=1)

weight = abs(np.mean(bandciCOH, axis = 2))

# weight > 0.1 as 
weight[np.where(weight<lowweight)] =0

In [96]:
graph = graphVisual.graph_create(weight, vsarea = elec_area, vscoord = elec_coord)

# set graph visualization style
visual_style = graphVisual.graph_style(graph)

In [97]:
title = 'Alpha normal'
texts[title] = [300, 225, 25]

igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)

igplot.show()

if 'title' in locals():
    del texts[title], title

### mild

In [98]:
lowweight = 0.1
bandciCOH = ciCOH_alpha_mild[0:128, 0:128, :]

filename = 'ciCOH_alpha_mild' + '.png'

savefile = os.path.join(savepath, filename)

In [99]:
# delete the unused channels in weight
bandciCOH = np.delete(bandciCOH, idx_notused, axis=0)
bandciCOH = np.delete(bandciCOH, idx_notused, axis=1)

weight = abs(np.mean(bandciCOH, axis = 2))

# weight > 0.1 as 
weight[np.where(weight<lowweight)] =0

In [100]:
graph = graphVisual.graph_create(weight, vsarea = elec_area, vscoord = elec_coord)

# set graph visualization style
visual_style = graphVisual.graph_style(graph)


In [101]:
title = 'Alpha mild'
texts[title] = [335, 225, 25]

igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
igplot.show()

if 'title' in locals():
    del texts[title], title

## Beta Visual

###  beta1

#### normal

In [102]:
del bandciCOH

bandciCOH = ciCOH_beta1_normal[0:128, 0:128, :]

filename = 'ciCOH_beta1_normal' + '.png'

savefile = os.path.join(savepath, filename)

# delete the unused channels in weight
bandciCOH = np.delete(bandciCOH, idx_notused, axis=0)
bandciCOH = np.delete(bandciCOH, idx_notused, axis=1)

In [103]:
weight = abs(np.mean(bandciCOH, axis = 2))

# weight > 0.1 as 
weight[np.where(weight<lowweight)] =0

In [104]:
graph = graphVisual.graph_create(weight, vsarea = elec_area, vscoord = elec_coord)

# set graph visualization style
visual_style = graphVisual.graph_style(graph)

In [105]:
title = 'Beta1 normal'
texts[title] = [335, 225, 25]

igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
igplot.show()

if 'title' in locals():
    del texts[title], title

#### mild

In [106]:
del bandciCOH

lowweight = 0.1
bandciCOH = ciCOH_beta1_mild[0:128, 0:128, :]

filename = 'ciCOH_beta1_mild' + '.png'

savefile = os.path.join(savepath, filename)

# delete the unused channels in weight
bandciCOH = np.delete(bandciCOH, idx_notused, axis=0)
bandciCOH = np.delete(bandciCOH, idx_notused, axis=1)

In [107]:
weight = abs(np.mean(bandciCOH, axis = 2))

# weight > 0.1 as 
weight[np.where(weight<lowweight)] =0

In [108]:
graph = graphVisual.graph_create(weight, vsarea = elec_area, vscoord = elec_coord)

# set graph visualization style
visual_style = graphVisual.graph_style(graph)

In [109]:
title = 'Beta1 mild'
texts[title] = [335, 225, 25]

igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
igplot.show()

if 'title' in locals():
    del texts[title], title

### beta2

#### normal

In [110]:
del bandciCOH

bandciCOH = ciCOH_beta2_normal[0:128, 0:128, :]

filename = 'ciCOH_beta2_normal' + '.png'

savefile = os.path.join(savepath, filename)

# delete the unused channels in weight
bandciCOH = np.delete(bandciCOH, idx_notused, axis=0)
bandciCOH = np.delete(bandciCOH, idx_notused, axis=1)

In [111]:
weight = abs(np.mean(bandciCOH, axis = 2))

# weight > 0.1 as 
weight[np.where(weight<lowweight)] =0

In [112]:
graph = graphVisual.graph_create(weight, vsarea = elec_area, vscoord = elec_coord)

# set graph visualization style
visual_style = graphVisual.graph_style(graph)

In [113]:
title = 'Beta2 normal'
texts[title] = [335, 225, 25]

igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
igplot.show()

if 'title' in locals():
    del texts[title], title

#### mild

In [114]:
if 'bandciCOH' in locals():
    del bandciCOH

bandciCOH = ciCOH_beta2_mild[0:128, 0:128, :]

if 'filename' in locals():
    del filename

filename = 'ciCOH_beta2_mild' + '.png'

savefile = os.path.join(savepath, filename)

# delete the unused channels in weight
bandciCOH = np.delete(bandciCOH, idx_notused, axis=0)
bandciCOH = np.delete(bandciCOH, idx_notused, axis=1)

In [115]:
weight = abs(np.mean(bandciCOH, axis = 2))

# weight > 0.1 as 
weight[np.where(weight<lowweight)] =0

In [116]:
graph = graphVisual.graph_create(weight, vsarea = elec_area, vscoord = elec_coord)

# set graph visualization style
visual_style = graphVisual.graph_style(graph)

In [117]:
title = 'Beta2 mild'
texts[title] = [335, 225, 25]

igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
igplot.show()

if 'title' in locals():
    del texts[title], title